In [1]:
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive
from IPython.display import clear_output, display, HTML

In [3]:
import numpy as np
from scipy import integrate

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation
from matplotlib.pyplot import figure

In [4]:
import bqplot

In [5]:
from IPython.display import YouTubeVideo

In [6]:
fsz = 10

In [7]:
YouTubeVideo('fma5WBl90Sw', width=800, height=(9/16)*800, mute=True)

# Frottements visqueux laminaires
![laminar](cylinder_laminar.jpg)

## Equation : 

$\vec{F_f} = -k \cdot \vec{v}$

avec:

$k = 6 \pi \mu r$

*pour une sphère à vitesse modérée*

## Série 6 ex. 3

![exercise](s6ex3.png)

## Somme des forces
Forces sur le point materiel :
$ \sum \vec{F} = m \cdot \vec{a} $

2 forces : 

$\vec{F_f} = - k \cdot \vec{v}$

$\vec{P} = - \vec{e_y} \cdot g \cdot m$

## Mouvement purement vertical donc:

$ \sum F_z = -m\cdot g - k \cdot v_z = m \cdot a_Z$

$ -m\cdot g - k \cdot \dot{z} = m \cdot \ddot{z}$

$ \ddot{z} + \frac{k}{m} \cdot \dot{z} = - g $

$ \ddot{z} = - g - \frac{k}{m} \cdot \dot{z} $

In [8]:
def a(v,t,g,k,m):
    return -g-(k/m)*v

In [9]:
def solve_state_space(g,k,m,v0,tstop = 10,samples = 300,plotMin=False):
    plot=True
    t = np.linspace(0, tstop, samples)
    
    solV = integrate.odeint(a, v0, t, args=(g, k, m))
    solZ = np.cumsum(solV)*(tstop/samples)
    
    if plot:
        fig, ax1 = plt.subplots(figsize=(fsz,fsz))
        
        color = 'tab:red'
        ax1.set_xlabel('time (s)')
        ax1.set_ylabel('speed', color=color)
        ax1.set_xlim([-1.,tstop])
        ax1.set_ylim([-40.,60.])
        ax1.plot(t, solV, color=color)
        ax1.tick_params(axis='y', labelcolor=color)

        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

        color = 'tab:blue'
        ax2.set_ylabel('position', color=color)  # we already handled the x-label with ax1
        ax2.plot(t, solZ, color=color)
        ax2.set_xlim([-1.,tstop])
        ax2.set_ylim([-40.,60.])
        ax2.plot([-100.,tstop],[0,0],'black',lw=1)
        ax2.tick_params(axis='y', labelcolor=color)
        
        if plotMin:
            plt.axvline(x=t[np.argmin(np.abs(solV))],color="green")
            zmax = solZ[np.argmin(np.abs(solV))]
            ax2.plot([-100.,tstop],[zmax,zmax],'green',lw=1)
    
    return (solZ,solV)

In [10]:
def plot_state_space(g,k,m,stream = False):
    
    X, Y = np.meshgrid(np.arange(-20, 20, 2), np.arange(-20, 20, 2))

    x_shape = X.shape
    
    U = np.zeros(x_shape)
    V = np.zeros(x_shape)

    for x in range(x_shape[0]):
        for y in range(x_shape[1]):
            U[x,y] = Y[x,y]
            V[x,y] = a(Y[x,y],0,g,k,m)

    fig, ax = plt.subplots(figsize=(fsz,fsz))
    #fig= plt.figure(figsize=(15,15))
    if stream:
        q = ax.streamplot(X, Y, U, V)
    else:
        q = ax.quiver(X, Y, U, V, units='xy' ,scale=None, color='red')
    
    ax.set_xlabel('position')
    ax.set_ylabel('speed')
    #sol = solve_state_space(g,k,m,v0,False,tstop,samples)
    #plt.plot(-sol[0],sol[1])
    ax.set_aspect('equal')

    plt.xlim(-22,22)
    plt.ylim(-22,22)

    plt.show()

## Solution (analytique) de l'équation différencielle

$ \ddot{z} + \frac{k}{m} \cdot \dot{z}  = - g $

$\Rightarrow$ **deux sous-solutions : la solution particulière et la solution homogène**

*La solution complète est une combinaison linéaire des deux autres*


### Solution Homogène

$\Rightarrow \ddot{z} + \frac{k}{m} \cdot \dot{z}(t) = 0$

on cherche une fonction pour $v(t)$ dont la dérivée est égale a elle meme multipliée par une constante arbitraite

$f(t)= e^{at}$

$f'(t) = a \cdot e^{at}$

hypothèse $v(t)= \dot{z} = e^{at}$ 

alors $\frac{dv(t)}{dt}=\ddot{z}= a \cdot e^{at} = a \cdot v(t)$

$\Rightarrow \frac{dv(t)}{dt}=\ddot{z}= a \cdot e^{at} = a \cdot v(t) = -\frac{b}{m} \cdot v(t)$

car

$\ddot{z} = - \frac{k}{m} \cdot \dot{z}(t)$

On a donc la solution homogène : $v_{homogene}(t) = CST \cdot e^{-\frac{b}{m}t}$

### Solution Particulière

**Supposons que la vitesse soit une constante**

$\Rightarrow \frac{dv(t)}{dt} = \ddot{z} = 0$

$\frac{dv(t)}{dt} + \frac{k}{m}*v_{part} = -g$

*devient donc*

$0 + \frac{k}{m}*v_{part} = -g$

$v_{part} = \frac{-mg}{b}$

*ici pas de constante*

### Solution pour v(t) (combinaison linéare des deux autres solutions)

$v(t) = v_{part}  + CST \cdot v_{homogene}(t)$

$v(t) =\frac{-mg}{b} + CST e^{-\frac{b}{m}t}$

$\Rightarrow$ conditions initiales :

$v(t=0)=v_0$

$z(t=0)=0$

*Donc:*

$v(t=0)=v_0 =\frac{-mg}{b} + CST e^{-\frac{b}{m}0}$

$v_0 + \frac{mg}{b} = CST *1$

*D'où on trouve la solution*

$v(t) = \frac{-mg}{b}  + (v_0 + \frac{mg}{b}) \cdot e^{-\frac{b}{m}t}$

### Remarques sur la solution v(t)

$v(t) = v_{part}  + (v_0 + \frac{mg}{b}) \cdot v_{homogene}(t)$

$\lim_{t \rightarrow \infty} v_{homogene}(t) = \lim_{t \rightarrow \infty}  e^{-\frac{b}{m}t} = 0$

$\lim_{t \rightarrow \infty} v(t) = v_{part}$

$\lim_{t \rightarrow \infty} v(t) = v_{part} + \lim_{t \rightarrow \infty} (v_0 + \frac{mg}{b}) \cdot v_{homogene}(t) = v_{part} + 0 = v_{part}$

### Solution z(t)

On a $v(t) = \dot{z(t)} = \frac{dz(t)}{dt}$

Et donc $z(t) = \int v(t) dt + CST$

$z(t) = \int [ \frac{-mg}{b}  + (v_0 + \frac{mg}{b}) \cdot e^{-\frac{b}{m}t}] dt + CST$

$z(t) = \int  \frac{-mg}{b} dt +  (v_0 + \frac{mg}{b}) \int e^{-\frac{b}{m}t} dt + CST$

$z(t) = \frac{-mg}{b} \cdot t +  (v_0 + \frac{mg}{b})  \cdot \frac{-m}{b} \cdot (e^{-\frac{b}{m}t} - 1) + CST$

$z(t) = \frac{-mg}{b} \cdot t +  (v_0 + \frac{m^2g}{b^2})  \cdot (1 - e^{-\frac{b}{m}t}) + CST$

Quelle est la valeur de $CST$ ????

$z(t=0) = 0$

$\Rightarrow \frac{-mg}{b} \cdot 0 +  (v_0 + \frac{m^2g}{b^2})  \cdot (1 - e^{-\frac{b}{m}0}) + CST = 0$

$\Rightarrow 0 +  (v_0 + \frac{m^2g}{b^2})  \cdot (0) + CST = 0$

$\Rightarrow CST = 0$

Donc on  a le système décrit par:

$z(t) = \frac{-mg}{b} \cdot t +  (v_0 + \frac{m^2g}{b^2})  \cdot (1 - e^{-\frac{b}{m}t})$

$v(t) = \frac{-mg}{b}  + (v_0 + \frac{mg}{b}) \cdot e^{-\frac{b}{m}t}$

# C la teuf

$z(t) = \frac{-mg}{b} \cdot t +  (v_0 + \frac{m^2g}{b^2})  \cdot (1 - e^{-\frac{b}{m}t})$

$v(t) = \frac{-mg}{b}  + (v_0 + \frac{mg}{b}) \cdot e^{-\frac{b}{m}t}$

In [11]:
w = interactive(solve_state_space, g=(0.,20.), k=(0., 30.0),
            m=(0.,50.),v0=(-5,80),tstop=(0.2,30.),samples=(1,500),plotMin=False)
display(w)

interactive(children=(FloatSlider(value=10.0, description='g', max=20.0), FloatSlider(value=15.0, description=…

## State-Space
State S given by 

$S = \begin{pmatrix}z\\ \dot{z}\end{pmatrix}$

It's derivative given by : 

$\dot{S} = \begin{pmatrix}\dot{z}\\ \ddot{z}\end{pmatrix} = \begin{pmatrix}\dot{z}\\ - g - \frac{k}{m} \cdot \dot{z}  \end{pmatrix} $

In [12]:
w = interactive(plot_state_space, g=(0.,20.), k=(0., 30.0),
            m=(0.,200.))
display(w)

interactive(children=(FloatSlider(value=10.0, description='g', max=20.0), FloatSlider(value=15.0, description=…